In [ ]:

#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json" 
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json" 

#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

#r "nuget: FSharp.Data"

open FSharp.Data

let wb = WorldBankData.GetDataContext()
let br = wb.Countries.``Brazil``

let getInterval indicator starting ending =
    seq {
        for year, value in indicator do
            yield year, value
    }
    |> Seq.filter(fun (year,_) -> year >= starting && year <= ending)
    |> Seq.toList
    |> List.unzip

open XPlot.Plotly

let getScatter title (year, value) =
    Scatter( name = title, x = year, y = value)

"Ready"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages FSharp.Data, 4.2.8 XPlot.Plotly.Interactive, 4.0.6

Ready

In [ ]:
let getGdp starting ending title = 
    let gdp = br.Indicators.``GDP growth (annual %)``
    getInterval gdp starting ending |> getScatter title
seq {
    yield getGdp 2003 2011 "Lula"
    yield getGdp 2011 2016 "Dilma"
    yield getGdp 2016 2018 "Temer"
    yield getGdp 2018 2022 "Bolsonaro"
}
|> Chart.Plot
|> Chart.WithTitle "GDP growth (annual %)"

<!DOCTYPE html>

In [ ]:
let getInflation starting ending title = 
    let gdp = br.Indicators.``Inflation, consumer prices (annual %)``
    getInterval gdp starting ending |> getScatter title
seq {
    yield getInflation 2003 2011 "Lula"
    yield getInflation 2011 2016 "Dilma"
    yield getInflation 2016 2018 "Temer"
    yield getInflation 2018 2022 "Bolsonaro"
}
|> Chart.Plot
|> Chart.WithTitle "Inflação, preços ao consumidor"

<!DOCTYPE html>

In [ ]:
let getUnemployment starting ending title = 
    let gdp = br.Indicators.``Unemployment, total (% of total labor force) (modeled ILO estimate)``
    getInterval gdp starting ending |> getScatter title
seq {
    yield getUnemployment 2003 2011 "Lula"
    yield getUnemployment 2011 2016 "Dilma"
    yield getUnemployment 2016 2018 "Temer"
    yield getUnemployment 2018 2022 "Bolsonaro"
}
|> Chart.Plot
|> Chart.WithTitle "Desemprego"

<!DOCTYPE html>